In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from s_dbw import S_Dbw
import math

In [2]:
# resets index after selections have been made
def resetindex(data):
    data=data.reset_index()
    data=data.drop('index', axis=1)
    return data

In [3]:
#reduces the data set to the first m iterations
def firstiterations(m,data):
    i=0
    j=0
    n=len(data.index)    
    while i<n:
        if  (data.iat[j,10]>m):
            data=data.drop(i, axis=0)
            i=i+1
        else:
            i=i+1
            j=j+1
    data=resetindex(data)
    return data

In [4]:
# Generate a Series with a specific length, so that all residuals of one parameter combination are in one line
def GenerateSeries(source, Length):
    output=source.iloc[:,[4,5]]
    output=output.drop_duplicates()
    output=resetindex(output)
    for j in range(1, Length+1):
        output[f'ResidualNorm{j}']=0.0
        for i in range(len(output.index)):
            Abfrage=source.query( f"`Droptol`=={output.iat[i,0]} and `Fillfactor`=={output.iat[i,1]} and Iterations=={j}")
            if len(Abfrage.index)==1:
                output.iat[i,j+1]=Abfrage.iat[0,8]
            elif len(Abfrage.index)>1:
                raise Exception("Multiple instances with same Droptol, Fillfactor and Iteration within one Beta")
    return output

In [266]:
df=pd.read_csv('PCGDim16129.txt', sep='\t')
df=df.drop('Convergence Mode', axis=1)

beta=1e0
dfBeta=df.query(f"Beta=={beta}")
dfBeta=resetindex(dfBeta)

In [267]:
test=dfBeta[["Droptol","Fillfactor"]]
test=test.drop_duplicates()
test=resetindex(test)
test

,Droptol,Fillfactor
0,1.000000e-12,0
1,1.000000e-12,1
2,1.000000e-12,2
3,1.000000e-12,3
4,1.000000e-12,4
...,...,...
85,1.000000e+04,5
86,1.000000e+04,6
87,1.000000e+04,7
88,1.000000e+04,8


In [268]:
print(test.iloc[0,:])

Droptol       1.000000e-12
Fillfactor    0.000000e+00
Name: 0, dtype: float64


In [269]:
Daten=dfBeta.query("Iterations==9")
Daten=resetindex(Daten)
Daten

,ILUTNonzeros,Number of unknowns,Alpha,Beta,Droptol,Fillfactor,Tolerance,InitialResidualNorm,ResidualNorm,ResidualQuotient,Iterations,SolutionTime,ILUAssemblyTime,AbsoluteError,RelativeError
0,16129,16129,1,1.0,1.000000e-12,0,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.006730,0.012128,0.168246,0.828086
1,53037,16129,1,1.0,1.000000e-12,1,1.000000e-14,1.933380e-07,0.000004,21.7681,9,0.008189,0.013694,0.168896,0.831286
2,124530,16129,1,1.0,1.000000e-12,2,1.000000e-14,1.933380e-07,0.000003,13.7632,9,0.007294,0.040581,0.128591,0.632911
3,158914,16129,1,1.0,1.000000e-12,3,1.000000e-14,1.933380e-07,0.000002,10.1783,9,0.007423,0.049023,0.102859,0.506261
4,200321,16129,1,1.0,1.000000e-12,4,1.000000e-14,1.933380e-07,0.000002,11.4695,9,0.008553,0.063150,0.063122,0.310678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,16129,16129,1,1.0,1.000000e+04,5,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.003870,0.008721,0.168246,0.828086
86,16129,16129,1,1.0,1.000000e+04,6,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.003133,0.008032,0.168246,0.828086
87,16129,16129,1,1.0,1.000000e+04,7,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.004053,0.009243,0.168246,0.828086
88,16129,16129,1,1.0,1.000000e+04,8,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.006148,0.011465,0.168246,0.828086


In [270]:
Daten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ILUTNonzeros         90 non-null     int64  
 1   Number of unknowns   90 non-null     int64  
 2   Alpha                90 non-null     int64  
 3   Beta                 90 non-null     float64
 4   Droptol              90 non-null     float64
 5   Fillfactor           90 non-null     int64  
 6   Tolerance            90 non-null     float64
 7   InitialResidualNorm  90 non-null     float64
 8   ResidualNorm         90 non-null     float64
 9   ResidualQuotient     90 non-null     float64
 10  Iterations           90 non-null     int64  
 11  SolutionTime         90 non-null     float64
 12  ILUAssemblyTime      90 non-null     float64
 13  AbsoluteError        90 non-null     float64
 14  RelativeError        90 non-null     float64
dtypes: float64(10), int64(5)
memory usage: 10.

In [271]:
for i in range(0,90):
    print(i)
    temp=Daten.query(f"Droptol=={test.iloc[i,0]} and Fillfactor=={test.iloc[i,1]}")
    temp=resetindex(temp)
    #print(len(temp.index))
    if len(temp.index)==0:
        temp2=(dfBeta.query(f"Droptol=={test.iloc[i,0]} and Fillfactor=={test.iloc[i,1]}")).tail(1)
        print(temp2)
        Daten.loc[len(Daten.index)] = [temp2.iloc[0,0], temp2.iloc[0,1],temp2.iloc[0,2],temp2.iloc[0,3],temp2.iloc[0,4],temp2.iloc[0,5],temp2.iloc[0,6],temp2.iloc[0,7],0.0,0.0,9,temp2.iloc[0,11],temp2.iloc[0,12],temp2.iloc[0,13],temp2.iloc[0,14]]
        #print(Daten.iloc[len(Daten.index),:])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [272]:
Daten

,ILUTNonzeros,Number of unknowns,Alpha,Beta,Droptol,Fillfactor,Tolerance,InitialResidualNorm,ResidualNorm,ResidualQuotient,Iterations,SolutionTime,ILUAssemblyTime,AbsoluteError,RelativeError
0,16129,16129,1,1.0,1.000000e-12,0,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.006730,0.012128,0.168246,0.828086
1,53037,16129,1,1.0,1.000000e-12,1,1.000000e-14,1.933380e-07,0.000004,21.7681,9,0.008189,0.013694,0.168896,0.831286
2,124530,16129,1,1.0,1.000000e-12,2,1.000000e-14,1.933380e-07,0.000003,13.7632,9,0.007294,0.040581,0.128591,0.632911
3,158914,16129,1,1.0,1.000000e-12,3,1.000000e-14,1.933380e-07,0.000002,10.1783,9,0.007423,0.049023,0.102859,0.506261
4,200321,16129,1,1.0,1.000000e-12,4,1.000000e-14,1.933380e-07,0.000002,11.4695,9,0.008553,0.063150,0.063122,0.310678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,16129,16129,1,1.0,1.000000e+04,5,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.003870,0.008721,0.168246,0.828086
86,16129,16129,1,1.0,1.000000e+04,6,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.003133,0.008032,0.168246,0.828086
87,16129,16129,1,1.0,1.000000e+04,7,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.004053,0.009243,0.168246,0.828086
88,16129,16129,1,1.0,1.000000e+04,8,1.000000e-14,1.933380e-07,0.000002,12.4683,9,0.006148,0.011465,0.168246,0.828086


In [273]:
def black_box_function(x, y):

    exp=round(x/2)*2
    xd=10**exp

    yd=round(y, 0) #Fillfactor
    resnorm=-(Daten.query(f"`Droptol`=={xd} and `Fillfactor`=={yd}")).iat[0,8]
    print(resnorm,'\t', xd,'\t', yd)
    f = open(f"BayesOpt1e{int(math.log10(beta))}.txt", "a")
    f.write(f"{resnorm}\t{xd}\t{yd}\n")
    f.close()
    return resnorm

In [274]:
from bayes_opt import BayesianOptimization

In [275]:
# Bounded region of parameter space
pbounds = {'x': (-12, +4), 'y': (0, 9)}

In [276]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [277]:
optimizer.probe(
    params={"x": -12, "y": 9},
    lazy=True,
)


optimizer.probe(
    params={"x": -4, "y": 9},
    lazy=True,
)

optimizer.probe(
    params={"x": +4, "y": 9},
    lazy=True,
)


optimizer.probe(
    params={"x": -12, "y": 4},
    lazy=True,
)

optimizer.probe(
    params={"x": -4, "y": 4},
    lazy=True,
)

optimizer.probe(
    params={"x": +4, "y": 4},
    lazy=True,
)


optimizer.probe(
    params={"x": -12, "y": 0},
    lazy=True,
)

optimizer.probe(
    params={"x": -4, "y": 0},
    lazy=True,
)

optimizer.probe(
    params={"x": +4, "y": 0},
    lazy=True,
)


In [278]:
optimizer.maximize(
    init_points=0,
    n_iter=200,
    alpha=0
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
-1.39233e-08 	 1e-12 	 9
|  1        | -1.392e-0 | -12.0     |  9.0      |
-1.38553e-08 	 0.0001 	 9
|  2        | -1.386e-0 | -4.0      |  9.0      |
-2.41059e-06 	 10000 	 9
|  3        | -2.411e-0 |  4.0      |  9.0      |
-2.21749e-06 	 1e-12 	 4
|  4        | -2.217e-0 | -12.0     |  4.0      |
-2.21766e-06 	 0.0001 	 4
|  5        | -2.218e-0 | -4.0      |  4.0      |
-2.41059e-06 	 10000 	 4
|  6        | -2.411e-0 |  4.0      |  4.0      |
-2.41059e-06 	 1e-12 	 0
|  7        | -2.411e-0 | -12.0     |  0.0      |
-2.41059e-06 	 0.0001 	 0
|  8        | -2.411e-0 | -4.0      |  0.0      |
-2.41059e-06 	 10000 	 0
|  9        | -2.411e-0 |  4.0      |  0.0      |
-1.39233e-08 	 1e-08 	 9.0
|  10       | -1.392e-0 | -7.923    |  8.98     |
-1.39176e-08 	 1e-06 	 9.0
|  11       | -1.392e-0 | -5.736    |  8.99     |
-1.39233e-08 	 1e-10 	 9.0
|  12       | -1.392e-0 | -10.29    |  8.

-2.27192e-07 	 1e-12 	 7.0
|  77       | -2.272e-0 | -11.4     |  7.328    |
-2.80769e-07 	 0.01 	 9.0
|  78       | -2.808e-0 | -1.052    |  8.662    |
-3.66007e-08 	 0.0001 	 8.0
|  79       | -3.66e-08 | -3.731    |  7.538    |
-2.27192e-07 	 1e-10 	 7.0
|  80       | -2.272e-0 | -9.854    |  6.805    |
-2.41059e-06 	 1e-10 	 0.0
|  81       | -2.411e-0 | -9.888    |  0.01054  |
-6.04419e-07 	 1e-08 	 6.0
|  82       | -6.044e-0 | -7.277    |  5.703    |
-6.05181e-07 	 0.0001 	 6.0
|  83       | -6.052e-0 | -3.954    |  6.417    |
-5.38776e-07 	 0.01 	 7.0
|  84       | -5.388e-0 | -2.374    |  6.863    |
-2.41059e-06 	 1e-06 	 0.0
|  85       | -2.411e-0 | -6.166    |  0.02014  |
-2.66094e-06 	 1e-12 	 2.0
|  86       | -2.661e-0 | -11.99    |  2.223    |
-6.04419e-07 	 1e-12 	 6.0
|  87       | -6.044e-0 | -11.97    |  5.599    |
-2.27192e-07 	 1e-08 	 7.0
|  88       | -2.272e-0 | -8.194    |  7.076    |
-3.6453e-08 	 1e-08 	 8.0
|  89       | -3.645e-0 | -8.97     |  8.148    |


-1.66741e-06 	 0.0001 	 5.0
|  153      | -1.667e-0 | -4.895    |  4.982    |
-3.6453e-08 	 1e-10 	 8.0
|  154      | -3.645e-0 | -9.576    |  8.022    |
-1.7307e-06 	 0.01 	 4.0
|  155      | -1.731e-0 | -2.605    |  4.096    |
-1.39233e-08 	 1e-08 	 9.0
|  156      | -1.392e-0 | -7.414    |  8.627    |
-6.0442e-07 	 1e-06 	 6.0
|  157      | -6.044e-0 | -6.044    |  6.46     |
-3.6453e-08 	 1e-10 	 8.0
|  158      | -3.645e-0 | -10.69    |  8.152    |
-2.27192e-07 	 1e-08 	 7.0
|  159      | -2.272e-0 | -8.64     |  6.688    |
-3.6453e-08 	 1e-08 	 8.0
|  160      | -3.645e-0 | -8.502    |  8.12     |
-1.38553e-08 	 0.0001 	 9.0
|  161      | -1.386e-0 | -4.515    |  8.998    |
-2.27193e-07 	 1e-06 	 7.0
|  162      | -2.272e-0 | -6.659    |  7.065    |
-3.66007e-08 	 0.0001 	 8.0
|  163      | -3.66e-08 | -4.55     |  7.675    |
-1.39233e-08 	 1e-10 	 9.0
|  164      | -1.392e-0 | -10.73    |  8.994    |
-3.66007e-08 	 0.0001 	 8.0
|  165      | -3.66e-08 | -4.24     |  8.201    |
-

In [279]:
print(optimizer.max)

{'target': -1.38553e-08, 'params': {'x': -4.0, 'y': 9.0}}


In [280]:
print(optimizer.res)

[{'target': -1.39233e-08, 'params': {'x': -12.0, 'y': 9.0}}, {'target': -1.38553e-08, 'params': {'x': -4.0, 'y': 9.0}}, {'target': -2.41059e-06, 'params': {'x': 4.0, 'y': 9.0}}, {'target': -2.21749e-06, 'params': {'x': -12.0, 'y': 4.0}}, {'target': -2.21766e-06, 'params': {'x': -4.0, 'y': 4.0}}, {'target': -2.41059e-06, 'params': {'x': 4.0, 'y': 4.0}}, {'target': -2.41059e-06, 'params': {'x': -12.0, 'y': 0.0}}, {'target': -2.41059e-06, 'params': {'x': -4.0, 'y': 0.0}}, {'target': -2.41059e-06, 'params': {'x': 4.0, 'y': 0.0}}, {'target': -1.39233e-08, 'params': {'x': -7.922604342713891, 'y': 8.98022188939807}}, {'target': -1.39176e-08, 'params': {'x': -5.7358021532482635, 'y': 8.990380045414742}}, {'target': -1.39233e-08, 'params': {'x': -10.285796186161067, 'y': 8.957610276953927}}, {'target': -2.80769e-07, 'params': {'x': -1.994708519024897, 'y': 8.997895738089902}}, {'target': -1.39233e-08, 'params': {'x': -9.030911293968762, 'y': 8.996318842720761}}, {'target': -1.39233e-08, 'params